In [1]:
from os.path import splitext, join
import numpy as np
from  tqdm import tqdm
import h5py
import pandas as pd
import open3d as o3d
import random


In [4]:
# Select files
files = np.array(os.listdir('HeadPointsAndLabels'))
point_files = np.sort(files[[splitext(file)[1] == '.pts' for file in files]])
pid_files = np.sort(files[[splitext(file)[1] == '.seg' for file in files]])

In [75]:
# Get points
points = []
for file in tqdm(point_files[0:50]): 
    points.append(pd.read_csv(join('HeadPointsAndLabels',file),sep =' ').to_numpy())
points= np.array(points)

100%|██████████| 50/50 [00:02<00:00, 18.77it/s]


In [74]:
# Get pid
pid = []
for file in tqdm(pid_files[0:50]): 
    pid.append(pd.read_csv(join('HeadPointsAndLabels',file)).to_numpy().flatten())
pid = np.array(pid)

100%|██████████| 50/50 [00:00<00:00, 58.23it/s]


In [88]:
# Downsample points
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points[0])
# 2048 random points 
# pcd.points = o3d.utility.Vector3dVector(points[0][random.sample(range(points[0].shape[0]), 2048),:])

pcd = pcd.uniform_down_sample(every_k_points=int(points[0].shape[0]/2048))
#pcd = pcd.voxel_down_sample(voxel_size=10)
print(np.asarray(pcd.points).shape)
o3d.visualization.draw_geometries([pcd])

(2073, 3)


In [159]:
# Create h5 file
with h5py.File("ply_data_train.h5", "w") as train_file:
    train_file.create_dataset("data", data = points)
    train_file.create_dataset("pid", data = pid)
    train_file.create_dataset("label", data = np.array([0]))

In [115]:
# Read h5 file
with h5py.File("ply_data_train.h5", "r") as train_file:
    data = train_file['data']
    pid = train_file['pid']
    label = train_file['label']


In [86]:
def display_labels(cloud, labels):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(cloud)

    no_label_cloud, left_eye_cloud, right_eye_cloud, nose_cloud, mouth_cloud = [pcd.select_down_sample(list(np.where(labels==l)[0])) for l in [0,1,2,3,4]]

    no_label_cloud.paint_uniform_color([0, 255, 255])
    left_eye_cloud.paint_uniform_color([255, 0, 0])
    right_eye_cloud.paint_uniform_color([0, 255, 0])
    nose_cloud.paint_uniform_color([255, 0, 255])
    mouth_cloud.paint_uniform_color([255, 255, 0])
    o3d.visualization.draw_geometries([no_label_cloud, left_eye_cloud, right_eye_cloud, nose_cloud, mouth_cloud])

In [92]:
display_labels(points[16], pid[16])

In [91]:
indexes = [idx for idx in range(0, points[0].shape[0], int(points[0].shape[0]/2048))]
points[0][indexes]
pid[0][indexes]
display_labels(points[0][indexes], pid[0][indexes])

In [60]:
pd.DataFrame(pid[0][indexes], columns=['d'])['d'].value_counts()

0    1971
3      53
4      33
1      10
2       6
Name: d, dtype: int64